# Data Analysis and Visualization of Cursor Accuracy with Tremor Identification

This code was developed to analyze data and generate figures for "Evaluating Mouse Control and Usability Patterns Across Tremor and Non-Tremor Populations"

### Loading necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Combining Mouse Click Accuracy and Tremor Data

In [ ]:
# XY cursor targets for tasks
targets = pd.read_csv("./targets.csv")
targets = targets[['X', 'Y']]

# tremor status of participants
tremors = pd.read_csv('../survey analysis/survey_results_text.csv', skiprows=1, header=0)
tremors = tremors[tremors['participant_ID']!='SURVEY TEST']
tremors = tremors[['participant_ID', 'tremor_level']]

In [ ]:
# Create dataframe with distance of mouse click from target in pixels
off_targets = pd.DataFrame({'participant_ID': [], 'off_target_distance': []})

for i in range(50):
    if os.path.exists("./data/positions_" + str(i) + ".txt"):
        # Cursor Data
        cursor = pd.read_csv("./data/positions_" + str(i) + ".txt", sep='\t', header=None, names=['time', 'X', 'Y'])
        clicks = cursor[cursor['time'] == 'Mouse Click'].index
        clicks = cursor.loc[clicks-1]
        clicks.reset_index(inplace=True)
        clicks = clicks[['X', 'Y']]

        x_dis = clicks['X']-targets['X']
        y_dis = clicks['Y']-targets['Y']

        for j in range(len(x_dis)):
            off_targets.loc[len(off_targets)] = ['PT' + str(i).zfill(2), np.mean(np.sqrt(x_dis[j]**2 + y_dis[j]**2))]

# Merge the data
click_accuracy = pd.merge(tremors, off_targets, on='participant_ID')

## Mixed-Effects Model of Task Click Accuracy

In [ ]:
# Fit a linear mixed-effects model
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.formula.api as smf

model = smf.mixedlm("off_target_distance ~ C(tremor_level)", data=click_accuracy, groups=click_accuracy["participant_ID"])
result = model.fit()
print(result.summary())

In [ ]:
plt.figure(figsize=(3.5,2.5))

sns.boxplot(data=click_accuracy, x='tremor_level', y='off_target_distance', color='grey')
plt.ylim([0, 80])
plt.title('Off Target Distance by Tremor Level', fontsize=12)
plt.xlabel('Tremor Occurance')
plt.ylabel('Distance from Target (px)')
plt.tight_layout()
plt.savefig('distance_to_target.png', dpi=300, bbox_inches='tight')

## Mean Click Accuracy per Participant

### Combine Data Into One DataFrame

In [ ]:
mean_click_accuracy = click_accuracy.groupby(['participant_ID', 'tremor_level'], as_index=False)['off_target_distance'].mean()

In [ ]:
label_map = {
    'No': 'Never',
    'Sometimes': 'Sometimes',
    'Yes': 'Always'
}

plt.figure(figsize=(3.5,2.5))

click_accuracy['tremor_label'] = click_accuracy['tremor_level'].map(label_map)
sns.boxplot(data=click_accuracy, x='tremor_label', y='off_target_distance', color='grey')
plt.ylim([0, 80])
plt.title('Off Target Distance by Tremor Level', fontsize=12)
plt.xlabel('Tremor Occurance')
plt.ylabel('Distance from Target (px)')
plt.tight_layout()
plt.savefig('distance_to_target.png', dpi=300, bbox_inches='tight')

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# One-way ANOVA: Score by tremor_level
anova_model = ols('off_target_distance ~ C(tremor_level)', data=mean_click_accuracy).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print(anova_table)


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Run Tukey’s test
tukey = pairwise_tukeyhsd(endog=mean_click_accuracy['off_target_distance'],
                          groups=mean_click_accuracy['tremor_level'],
                          alpha=0.05)

# Print summary table
print(tukey.summary())